In [ ]:
import numpy as np
import pandas as pd

**neuron functions**

In [ ]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

def relu(z):
    return np.maximum(z, 0)

def exp(z):
    return np.exp(z)

def I(z): # identity function
    return z

**Loss/Cost functions**

In [ ]:
def MAPE(pred, tgt):
    N = tgt.shape[0]
    I = tgt > 0
    tgt = tgt[I]
    pred = pred[I]
    return np.sum(np.divide(np.abs(pred - tgt), tgt)) / N

**derivative functions**

In [ ]:
def d(a, f):
    """
    Inputs
        a: a matrix of activations, a = f(z) for some input matrix z
        f: a function such that f(z) = a
    What it does:
        determines a' = df(z)/dz, the derivative of a with respect to z.
        For example, if a = exp(z) then a' = a.
    Outputs:
        the derivative of a with respect to z
    """
    return (np.ones_like(a) if f == I else
           np.sign(a) if f == relu else
           a*(1 - a) if f == sigmoid else
           a if f == exp else ValueError('bad function type'))

def dMAPE(pred, tgt):
    N = tgt.shape[0]
    return np.divide( np.multiply(tgt, np.sign(pred - tgt)), N*(np.multiply(tgt, tgt) + np.finfo(float).eps))

**randomization functions**

In [ ]:
def Gauss(n, s):
    return n*(1 + np.random.randn()/s)

def asymptotic(n, s):
    return 1 - (1 - n)/(1 + np.random.randn()/s)

In [ ]:
class Network:
    def __init__(self, sizes, flavors, eta, alpha, batch_size, epochs, loss, max_inputs):
        np.random.seed(1729)
        self.sizes = sizes # number of nodes in each layer, including input layer
        self.sizes[0] = min(self.sizes[0], max_inputs)
        self.flavors = flavors # function executed by nodes in given layer, excluding input layer
        self.eta = eta
        self.alpha = alpha
        self.batch_size = batch_size
        self.epochs = epochs
        self.loss = loss
        self.max_inputs = max_inputs
        self.weights = self.__get_weights__()
        self.biases = self.__get_biases__()
        
    def __get_weights__(self):
        return [np.matrix(0.01*np.random.rand(fro, to)) 
                for (to, fro) in zip(self.sizes[1:], self.sizes[:-1])]

    def __get_biases__(self):
        return [np.matrix(0.01*np.random.rand(to, 1)).T for to in self.sizes[1:]]
    
    def predict(self, X):
        """
        Inputs
            X: a matrix containing rows of input vectors
        What it does:
            Ths is the feed forward algorithm, It iterates through the network object, 
            updating the input vector X at each layer of the network using the weights, 
            biases and node function
        Outputs:
            a list of matrices corresponding to the activations at each layer of
            the matrix. For example, the first matrix is the input matrix, the last
            matrix is the prediction matrix. Intermediate matrices are the
            activations output by the hidden layers
        """
        A = [X]
        for i in xrange(len(self.flavors)):
            W, B, F = self.weights[i], self.biases[i], self.flavors[i]
            X = F(X*W + B)
            A.append(X)
        return A
            
    def __backprop__(self, A, Y):
        """
        Inputs
            A: a list containing matrices of activations for each layer of the
            network for the current minibatch
            Y: a list of targets for the curent minibatch
        What it does:
            given a batch of data, finds the gradients of the loss function
            with respect to each of the weight and bias matrices
        Outputs:
            a list containing all partials with respect to weight matrices
            and a list containing all partials with respect to bias matrices
        """
        nabla_CB, nabla_CW = [], []
        delta = dMAPE(A[-1], Y)
        seno = np.ones_like(Y).T # row ones vector
        for i in xrange(1, len(A)):
            # calculate z-derivative of current activation
            da = d(A[-i], self.flavors[-i])
            delta_da = np.multiply(delta, da)
            nabla_CB.insert(0, seno*delta_da)
            nabla_CW.insert(0, A[-i-1].T*(delta_da))
            delta = (delta_da)*self.weights[-i].T
        return nabla_CB, nabla_CW
    
    def train(self, X_fit, Y_fit):
        """
        Inputs
            X_fit: the portion of the training feature matrix used for fitting
            Y_fit:  the portion of the training target matrix used for fitting
            X_val: the portion of the training feature matrix used for validation
        What it does:
            Iterates over epochs in which the entire fit data matrix is used to
            train the network. Controls the batch generation and selection within
            epochs. Feeds batches one at a time into the feedforward and
            backpropagation functions. Updates the network object's weights and
            biases with the negative gradients derived by back prop and scaled by
            the learning rate factor. Decays the learning rate each epoch.
        Outputs:
            an array of predictions for the validation data from the fully trained
            network
        """
        np.random.seed(1729)
        eta = self.eta # make a copy as we're gonna decay it over time
        for ep in xrange(self.epochs):
            n = X_fit.shape[0]
            indices = np.random.shuffle([i for i in xrange(n)])
            batches = n / self.batch_size + 1 # cld --> ceiling division
            batch_size = 1.*n / batches # the used batch size is not precisely what
                                     # was requested, but optimal given the
                                     # amount of data
            k = 0
            while k < batches:
                lo = int(k*batch_size)
                hi = int((k+1)*batch_size)
                A_list = self.predict(X_fit[lo:hi,:])
                nabla_CB, nabla_CW = self.__backprop__(A_list, Y_fit[lo:hi])
                self.weights = ([w - eta*dCw for (w, dCw) in zip(self.weights, nabla_CW)])
                self.biases = ([b - eta*dCb for (b, dCb) in zip(self.biases, nabla_CB)])
                k += 1
            eta *= self.alpha

    def HyparamGen(self):
        np.random.seed()
        self.sizes = [int(Gauss(sz, 10)) for sz in self.sizes[0:-1]] + [1] # 1 on end
        self.sizes[0] = min(self.sizes[0], self.max_inputs)
        self.eta = Gauss(self.eta, 10)
        self.alpha = asymptotic(self.alpha, 10)
        self.epochs = int(Gauss(self.epochs, 10))
        np.random.seed(1729)
        self.weights = self.__get_weights__()
        self.biases = self.__get_biases__()

In [ ]:
def top_features(X, Y):
    """
    Inputs
        X: the full feature matrix
        Y: the full target matrix
    What it does:
        Finds the top features (columns) in the training feature matrix according to 
        correlation to the training target matrix.
    Outputs:
        the indices of the feature columns in order of correlation importance
    """
    lyste = abs(X.T * Y)
    indices = []
    for _ in xrange(len(lyste)):
        i = np.argmax(lyste)
        lyste[i] = 0.
        indices.append(i)
    return indices

In [ ]:
def CrossValidation(net, X_train, Y_train, X_test, Y_test, CV):
    """
    Inputs
        net: a neural network object
        X_train: the training feature matrix
        Y_train: the training target matrix
        X_test: the test feature matrix
        Y_test: the test target matrix
        CV: an array of arrays containg the fit and train indices for each
        cross-validation fold
    What it does:
        Iterates over cross-validation folds, splitting the data into
        appropriate fit and validation sets, and passing the fit and val sets
        on to the train function. Assembles validation predictions from the
        train function into a comprehensive list over the entire training data
        set. Assembles test data predictions from the feedforward function for
        every fold.
    Outputs:
        The validation loss, using whatever loss functon is defined for the
        network
        The mean of the test data predictions from each fold of the cross
        validation
    """
    psi = np.matrix(np.empty((0, 1)))
    Y = np.matrix(np.empty((0, 1)))
    psi_test = np.matrix(np.empty((X_test.shape[0], 0)))
    for in_fit, in_val in CV:
        X_fit, Y_fit = X_train[in_fit,:], Y_train[in_fit]
        X_val, Y_val = X_train[in_val,:], Y_train[in_val]
        #training
        net.train(X_fit, Y_fit)
        psi = np.vstack((psi, net.predict(X_val)[-1]))
        Y = np.vstack((Y, Y_val))
        psi_test = np.hstack((psi_test, net.predict(X_test)[-1]))
    return (net.loss(psi, Y), net.loss(np.mean(psi_test, axis=1), Y_test))

In [ ]:
from sklearn.cross_validation import train_test_split, KFold
import copy

def PerpetualSearch(net, X, Y, aktuellMAPE):
    folds = 3
    correlindices = top_features(X, Y)
    np.random.seed(1729)
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)
    CVfolds = KFold(Y_train.shape[0], folds, shuffle=True)
    optimal_net = copy.deepcopy(net)
    while True:
        print "doing something"
        # focus on top N features
        tops = correlindices[0:net.sizes[0]]
        X_train_tops, X_test_tops = X_train[:,tops], X_test[:,tops]
        currentMAPE, pseudo_testMAPE = CrossValidation(net, X_train_tops, Y_train, 
                                                       X_test_tops, Y_test, CVfolds)
        with open("Record.csv","a+") as f:
            for i in range(6):
                if i < len(net.sizes)-1:
                    f.write("%s\t" % net.sizes[i])
            f.write("%s\t" % net.eta)
            f.write("%s\t" % net.alpha)
            f.write("%s\t" %net.epochs)
            f.write("%s\t" % currentMAPE)
            f.write("%s\t" % pseudo_testMAPE)
            if currentMAPE < aktuellMAPE:
                net.train(X_train_tops, Y_train)
                true_test_loss = net.loss(net.predict(X_test_tops)[-1], Y_test)
                f.write("%s\n" % true_test_loss)  
                optimal_net = copy.deepcopy(net)
                aktuellMAPE = currentMAPE
            else:
                f.write("\n")
        net = copy.deepcopy(optimal_net)
        net.HyparamGen()

# Classic Grid Search. 

Do this first

In [ ]:
def GridSearch(X,Y, storageloc):
    folds = 3
    correlindices = top_features(X, Y)
    np.random.seed(1729)
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)
    CVfolds = KFold(Y_train.shape[0], folds, shuffle=True)
    aktuellMAPE = np.infty

    for w in [[10,10,10,10], [20,20,20,20], [50,50,50,50], [100,100,100,100]]:
        for eta in [0.1, 0.5, 1]:
            for alpha in [0.999, 0.99, 0.9]:
                for eps in [100, 200, 300, 400]:
                    net = Network(
                                  w + [1],          # nodes per layer
                                  [relu, relu, relu, relu],     # activation functions at each hidden and output layer
                                  eta,                                     # learning rate
                                  alpha,                                    # learning rate decay
                                  2048,                                    # desired batch size
                                  eps,                                     # number of epochs
                                  MAPE,                                    # loss function
                                  X.shape[1]                               # max number of features
                                 )
                    print "doing something"
                    # focus on top N features
                    tops = correlindices[0:net.sizes[0]]
                    X_train_tops, X_test_tops = X_train[:,tops], X_test[:,tops]
                    currentMAPE, pseudo_testMAPE = CrossValidation(net, X_train_tops, Y_train, 
                                                           X_test_tops, Y_test, CVfolds)
                    with open(storageloc,"a+") as f:
                        for sz in net.sizes[:-1]:
                            f.write("%s\t" % sz)
                        f.write("%s\t" % net.eta)
                        f.write("%s\t" % net.alpha)
                        f.write("%s\t" %net.epochs)
                        f.write("%s\t" % currentMAPE)
                        f.write("%s\t" % pseudo_testMAPE)
                        if currentMAPE < aktuellMAPE:
                            net.train(X_train_tops, Y_train)
                            true_test_loss = net.loss(net.predict(X_test_tops)[-1], Y_test)
                            f.write("%s\n" % true_test_loss)  
                            optimal_net = copy.deepcopy(net)
                            aktuellMAPE = currentMAPE
                        else:
                            f.write("\n")

In [ ]:
training_data = pd.read_csv('TrainingData.csv')

Y = np.matrix(training_data.iloc[:,2].values).T
X = np.matrix(training_data.iloc[:,3:].values)

training_data = 0

In [ ]:
GridSearch(X,Y, "Record.csv")

# Optimize the best known hyperparameters using perpetual search after classic grid search has completed. 

Perpetual search will run indefinitely; you have to interrupt the kernel to sop it.

In [ ]:
# initialize network object as starting point for perpetual search
record = pd.read_csv('Record.csv', delimiter='\t')
i = np.argmin(record["Val MAPE"])
aktuellMAPE = record.iloc[i, 7]
net = Network(
              [int(n) for n in (record.iloc[i, 0:4].values)] + [1], # nodes per layer
              [relu, relu, relu, relu],         # activation functions at each hidden and output layer
              record.iloc[i, 4],                # learning rate
              record.iloc[i, 5],                # learning rate decay
              2048,                             # desired batch size
              int(record.iloc[i, 6]),           # number of epochs
              MAPE,                             # loss function
              X.shape[1]                        # max number of features
             )
net.HyparamGen()

PerpetualSearch(net, X, Y, aktuellMAPE)

# Check stability with respect to alternate train test splits

In [ ]:
p = 1729
while p < 1745:
    p += 1
    np.random.seed(p)
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)
    folds = 3
    CVfolds = KFold(Y_train.shape[0], folds, shuffle=True)
    record = pd.read_csv('Record.csv', delimiter='\t')
    i = np.argmin(record["Val MAPE"])
    net = Network(
                  [int(n) for n in (record.iloc[i, 0:4].values)] + [1], # nodes per layer
                  [relu, relu, relu, relu],         # activation functions at each hidden and output layer
                  record.iloc[i, 4],                # learning rate
                  record.iloc[i, 5],                # learning rate decay
                  2048,                             # desired batch size
                  int(record.iloc[i, 6]),           # number of epochs
                  MAPE,                             # loss function
                  X.shape[1]                        # max number of features
                 )
    print "doing something"
    # focus on top N features
    correlindices = top_features(X, Y)
    tops = correlindices[0:net.sizes[0]]
    X_train_tops, X_test_tops = X_train[:,tops], X_test[:,tops]
    currentMAPE, pseudo_testMAPE = CrossValidation(net, X_train_tops, Y_train, 
                                           X_test_tops, Y_test, CVfolds)
    with open("Stability.csv","a+") as f:
        for sz in net.sizes[:-1]:
            f.write("%s\t" % sz)
        f.write("%s\t" % net.eta)
        f.write("%s\t" % net.alpha)
        f.write("%s\t" %net.epochs)
        f.write("%s\t" % currentMAPE)
        f.write("%s\t" % pseudo_testMAPE)
        f.write("\n")

# Re-read in data for some more detailed analyses

In [ ]:
# re-initialize X and Y for the next few analyses. They require the index values districtID and UTC
X = Y = 0

In [ ]:
training_data = pd.read_csv('TrainingData.csv')

Y = np.matrix(training_data.iloc[:,2].values).T
X = np.matrix(training_data.iloc[:,:].values)

training_data = 0

np.random.seed(1729) # ensures test labels are identical to those in grid search
X_train,X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

index_train = X_train[:,:2]
index_test = X_test[:,:2]
X_train = X_train[:,3:]
X_test = X_test[:,3:]

# Investigate model performance by districtID

In [ ]:
record = pd.read_csv('Record.csv', delimiter='\t')
i = np.argmin(record["Val MAPE"])
net = Network(
              [int(n) for n in (record.iloc[i, 0:4].values)] + [1], # nodes per layer
              [relu, relu, relu, relu],         # activation functions at each hidden and output layer
              record.iloc[i, 4],                # learning rate
              record.iloc[i, 5],                # learning rate decay
              2048,                             # desired batch size
              int(record.iloc[i, 6]),           # number of epochs
              MAPE,                             # loss function
              X.shape[1]                        # max number of features
             )

In [ ]:
correlindices = top_features(X, Y)
tops = correlindices[:net.sizes[0]]
X_train_tops, X_test_tops = X_train[:,tops], X_test[:,tops]

net.train(X_train_tops, Y_train)
predictions = net.predict(X_test_tops)

In [ ]:
combi= np.hstack([index_test, predictions[-1], Y_test])

districts = [x for x in xrange(1,67)]
districtMAPE, averagegap = [], []
for dID in districts:
    print dID, \
          np.mean(combi[np.where(combi[:,0]==dID)[0]][:,3]), \
          MAPE(combi[np.where(combi[:,0]==dID)[0]][:,2], combi[np.where(combi[:,0]==dID)[0]][:,3])

# Investigate effects of removing Jan 1st data from training

In [ ]:
from sklearn.cross_validation import train_test_split, KFold
import copy

def PerpetualSearchNoJ1(net, X, Y, aktuellMAPE):
    folds = 3
    correlindices = top_features(X, Y)
    np.random.seed(1729)
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)
    noJ1 = np.where(X_train[:,1]>144)[0]
    Y_train = Y_train[noJ1]
    X_train = X_train[noJ1,3:]
    X_test = X_test[:,3:]
    CVfolds = KFold(Y_train.shape[0], folds, shuffle=True)
    optimal_net = copy.deepcopy(net)
    while True:
        print "doing something"
        # focus on top N features
        tops = correlindices[:net.sizes[0]]
        X_train_tops, X_test_tops = X_train[:,tops], X_test[:,tops]
        currentMAPE, pseudo_testMAPE = CrossValidation(net, X_train_tops, Y_train, 
                                                       X_test_tops, Y_test, CVfolds)
        with open("RecordNoJ1.csv","a+") as f:
            for i in range(6):
                if i < len(net.sizes)-1:
                    f.write("%s\t" % net.sizes[i])
            f.write("%s\t" % net.eta)
            f.write("%s\t" % net.alpha)
            f.write("%s\t" %net.epochs)
            f.write("%s\t" % currentMAPE)
            f.write("%s\t" % pseudo_testMAPE)
            if currentMAPE < aktuellMAPE:
                net.train(X_train_tops, Y_train)
                true_test_loss = net.loss(net.predict(X_test_tops)[-1], Y_test)
                f.write("%s\n" % true_test_loss)  
                optimal_net = copy.deepcopy(net)
                aktuellMAPE = currentMAPE
            else:
                f.write("\n")
        net = copy.deepcopy(optimal_net)
        net.HyparamGen()

In [ ]:
# initialize network object as starting point for perpetual search
record = pd.read_csv('Record.csv', delimiter='\t')
i = np.argmin(record["Val MAPE"])
net = Network(
              [int(n) for n in (record.iloc[i, 0:4].values)] + [1], # nodes per layer
              [relu, relu, relu, relu],         # activation functions at each hidden and output layer
              record.iloc[i, 4],                # learning rate
              record.iloc[i, 5],                # learning rate decay
              2048,                             # desired batch size
              int(record.iloc[i, 6]),           # number of epochs
              MAPE,                             # loss function
              X.shape[1]                        # max number of features
             )

PerpetualSearchNoJ1(net, X, Y, np.infty)

# Guess-1 MAPE results

Here is the guess-1 result for the test data

In [ ]:
guess1 = np.ones_like(Y_test)
MAPE(guess1, Y_test)

Here is the guess- result for the validation (training) data

In [ ]:
guess1 = np.ones_like(Y_train)
MAPE(guess1, Y_train)

# Increase prior periods to 5

In [ ]:
X = Y = 0

In [ ]:
training_data5 = pd.read_csv('TrainingData5.csv')

Y = np.matrix(training_data5.iloc[:,2].values).T
X = np.matrix(training_data5.iloc[:,3:].values)

training_data5 = 0

In [ ]:
GridSearch(X,Y, "Record5.csv")

In [ ]:
# initialize network object as starting point for perpetual search
record = pd.read_csv('Record5.csv', delimiter='\t')
i = np.argmin(record["Val MAPE"])
aktuellMAPE = record.iloc[i, 7]
net = Network(
              [int(n) for n in (record.iloc[i, 0:4].values)] + [1], # nodes per layer
              [relu, relu, relu, relu],         # activation functions at each hidden and output layer
              record.iloc[i, 4],                # learning rate
              record.iloc[i, 5],                # learning rate decay
              2048,                             # desired batch size
              int(record.iloc[i, 6]),           # number of epochs
              MAPE,                             # loss function
              X.shape[1]                        # max number of features
             )

PerpetualSearch(net, X, Y, np.infty)